In [2]:
# !pip install mlxtend

  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/73/da/d5d77a9a7a135c948dbf8d3b873655b105a152d69e590150c83d23c3d070/mlxtend-0.23.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.4 MB 1.3 MB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.4 MB 2.4 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.4 MB 2.8 MB/s eta 0:00:01
   ---------------- ----------------------- 0.6/1.4 MB 2.8 MB/s eta 0:00:01
   --------------------- ------------------ 0.8/1.4 MB 2.9 MB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.4 MB 3.1 MB/s eta 0:00:01
   -------------------------------- ------- 1.2/1.4 MB 3.2 MB/s eta 0:00:01
   --------------------------------- ------ 1.2/1.4 MB 3.2 MB/s eta 0:00:01
   ------------------------------------- -- 1.4/1.4 MB

In [1]:
import pandas as pd
import numpy as np

### 연관규칙 분석을 위한 apriori 알고리즘 사용
from mlxtend.frequent_patterns import fpgrowth
### 연관 규칙 도출
from mlxtend.frequent_patterns import association_rules

In [5]:
### 데이터 데이터 불러오기
df_origin = pd.read_csv("./survey_total.csv")
df = df_origin.copy()

### 10~70대 연령대만 사용
# df = df[~df["연령대"].isin(["0~9","80~89","90~99"])].reset_index(drop=True)
df = df.iloc[:,3:]

df

,비타민 및 무기질,식이섬유,아미노산 및 단백질,지방산,엽록소,인삼류,페놀류,당 및 탄수화물,발효미생물류,신규기능성식품
0,1,0,0,0,0,0,0,0,1,1
1,1,0,0,1,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9555,0,0,0,0,0,1,0,0,1,0
9556,1,0,0,0,0,0,0,0,1,0
9557,0,0,0,0,0,0,0,0,1,0
9558,1,0,0,0,0,0,0,0,0,0


# FP-Growth 알고리즘

In [6]:
"""
*** 연관 규칙 수 조절 ***
- 최소지지도(min_support) : 0.001
 => 최소지지도는 전체(9천 건)에서 0.1%는 9개로 다품목 섭취 시 유의미할 것이라 판단

- 품목 집합 최대 개수(max_len) : 8
 => 희소한 조건도 포함시키기 위해 최대 품목 개수는 데이터 최대 개수인 8개로 설정
 => 1개:3442건 2개:4178건, 3개:1529건, 4개:328건, 5개:69건, 6개:6건, 7개:2건, 8개:4건
"""

### 지지도 도출 
# - 최소지지도 : 0.1%, 최대집합 수 : 8
frequent_itemsets = fpgrowth(df, min_support=0.001, use_colnames=True, max_len=8)

frequent_itemsets['length'] = frequent_itemsets['itemsets'].map(lambda x: len(x))
frequent_itemsets.sort_values('support',ascending=False,inplace=True)
frequent_itemsets

C:\Users\USER\anaconda3\envs\newenv\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets,length
0,0.600000,(비타민 및 무기질),1
1,0.404498,(발효미생물류),1
3,0.291632,(지방산),1
4,0.269456,(인삼류),1
10,0.210774,"(비타민 및 무기질, 발효미생물류)",2
...,...,...,...
57,0.001046,"(식이섬유, 신규기능성식품, 발효미생물류, 지방산)",4
64,0.001046,"(지방산, 비타민 및 무기질, 인삼류, 발효미생물류, 식이섬유)",5
107,0.001046,"(아미노산 및 단백질, 지방산, 당 및 탄수화물)",3
113,0.001046,"(식이섬유, 페놀류)",2


In [7]:
### 최소 신뢰도 0.001이상인것만 추출
association_rules_df = association_rules(frequent_itemsets,
                                         metric='confidence', 
                                         min_threshold=0.001)

# ### 최소 지지도 0.7이상인것만 추출
# rules = association_rules(frequent_itemsets,
#                           metric="lift", 
#                           min_threshold=0.7)

### 전체 신뢰도 : 사건이 모두 발생했을 경우
all_confidences = []
### 집합적 강도 : 연관 규칙의 성능 평가
collective_strengths = []
### 코사인 유사도
cosine_similarities = []

for _,row in association_rules_df.iterrows():
    ### if/then, 조건/결과 생성
    all_confidence_if = str(list(row['antecedents']))
    all_confidence_then = str(list(row['consequents']))
    # - 반대일 경우 순서 수정
    if row['antecedent support'] <= row['consequent support']:
        all_confidence_if = str(list(row['consequents']))
        all_confidence_then = str(list(row['antecedents']))
    
    ### 전체 신뢰도
    all_confidence = {all_confidence_if+' -> '+all_confidence_then : \
                      row['support']/max(row['antecedent support'], row['consequent support'])}
    all_confidences.append(all_confidence)
    
    ### 집합적 강도
    violation = row['antecedent support'] + row['consequent support'] - 2*row['support']
    ex_violation = 1-row['antecedent support']*row['consequent support'] - \
                    (1-row['antecedent support'])*(1-row['consequent support'])
    collective_strength = (1-violation)/(1-ex_violation)*(ex_violation/violation)
    collective_strengths.append(collective_strength)
    
    ### 코사인 유사도
    cosine_similarity = row['support']/np.sqrt(row['antecedent support']*row['consequent support'])
    cosine_similarities.append(cosine_similarity)
    
association_rules_df['all-confidence'] = all_confidences
association_rules_df['collective strength'] = collective_strengths
association_rules_df['cosine similarity'] = cosine_similarities
association_rules_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,all-confidence,collective strength,cosine similarity
0,(비타민 및 무기질),(발효미생물류),0.600000,0.404498,0.210774,0.351290,0.868460,-0.031925,0.917979,-0.274658,{'['비타민 및 무기질'] -> ['발효미생물류']': 0.351290097629...,0.772243,0.427842
1,(발효미생물류),(비타민 및 무기질),0.404498,0.600000,0.210774,0.521076,0.868460,-0.031925,0.835205,-0.202772,{'['비타민 및 무기질'] -> ['발효미생물류']': 0.351290097629...,0.772243,0.427842
2,(비타민 및 무기질),(지방산),0.600000,0.291632,0.162971,0.271618,0.931373,-0.012008,0.972523,-0.155556,{'['비타민 및 무기질'] -> ['지방산']': 0.27161785216178524},0.907368,0.389598
3,(지방산),(비타민 및 무기질),0.291632,0.600000,0.162971,0.558824,0.931373,-0.012008,0.906667,-0.094219,{'['비타민 및 무기질'] -> ['지방산']': 0.27161785216178524},0.907368,0.389598
4,(비타민 및 무기질),(인삼류),0.600000,0.269456,0.111925,0.186541,0.692288,-0.049749,0.898071,-0.526339,{'['비타민 및 무기질'] -> ['인삼류']': 0.18654114365411437},0.660457,0.278360
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"(식이섬유, 당 및 탄수화물)",(비타민 및 무기질),0.001255,0.600000,0.001046,0.833333,1.388889,0.000293,2.400000,0.280352,"{'['비타민 및 무기질'] -> ['식이섬유', '당 및 탄수화물']': 0.00...",1.002443,0.038116
996,"(비타민 및 무기질, 당 및 탄수화물)",(식이섬유),0.015167,0.032950,0.001046,0.068966,2.093049,0.000546,1.038684,0.530271,"{'['식이섬유'] -> ['비타민 및 무기질', '당 및 탄수화물']': 0.03...",1.024911,0.046791
997,(식이섬유),"(비타민 및 무기질, 당 및 탄수화물)",0.032950,0.015167,0.001046,0.031746,2.093049,0.000546,1.017122,0.540022,"{'['식이섬유'] -> ['비타민 및 무기질', '당 및 탄수화물']': 0.03...",1.024911,0.046791
998,(비타민 및 무기질),"(식이섬유, 당 및 탄수화물)",0.600000,0.001255,0.001046,0.001743,1.388889,0.000293,1.000489,0.700000,"{'['비타민 및 무기질'] -> ['식이섬유', '당 및 탄수화물']': 0.00...",1.002443,0.038116


In [11]:
association_rules_df.sort_values("lift",ascending=False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,all-confidence,collective strength,cosine similarity
887,"(인삼류, 발효미생물류, 비타민 및 무기질)","(페놀류, 지방산)",0.028415,0.008149,0.001179,0.041509,5.093644,0.000948,1.034805,0.827181,"{'['인삼류', '발효미생물류', '비타민 및 무기질'] -> ['페놀류', '지...",1.057502,0.077511
890,"(페놀류, 지방산)","(인삼류, 발효미생물류, 비타민 및 무기질)",0.008149,0.028415,0.001179,0.144737,5.093644,0.000948,1.136007,0.810280,"{'['인삼류', '발효미생물류', '비타민 및 무기질'] -> ['페놀류', '지...",1.057502,0.077511
901,(페놀류),"(인삼류, 지방산, 발효미생물류, 비타민 및 무기질)",0.026807,0.008793,0.001179,0.044000,5.004195,0.000944,1.036828,0.822208,"{'['페놀류'] -> ['인삼류', '지방산', '발효미생물류', '비타민 및 무...",1.058853,0.076827
876,"(인삼류, 지방산, 발효미생물류, 비타민 및 무기질)",(페놀류),0.008793,0.026807,0.001179,0.134146,5.004195,0.000944,1.123970,0.807266,"{'['페놀류'] -> ['인삼류', '지방산', '발효미생물류', '비타민 및 무...",1.058853,0.076827
884,"(지방산, 발효미생물류, 인삼류)","(페놀류, 비타민 및 무기질)",0.017371,0.014261,0.001179,0.067901,4.761255,0.000932,1.057548,0.803936,"{'['지방산', '발효미생물류', '인삼류'] -> ['페놀류', '비타민 및 무...",1.065707,0.074939
893,"(페놀류, 비타민 및 무기질)","(지방산, 발효미생물류, 인삼류)",0.014261,0.017371,0.001179,0.082707,4.761255,0.000932,1.071227,0.801400,"{'['지방산', '발효미생물류', '인삼류'] -> ['페놀류', '비타민 및 무...",1.065707,0.074939
971,"(식이섬유, 지방산)","(인삼류, 발효미생물류, 비타민 및 무기질)",0.007935,0.028415,0.001072,0.135135,4.755737,0.000847,1.123395,0.796044,"{'['인삼류', '발효미생물류', '비타민 및 무기질'] -> ['식이섬유', '...",1.051356,0.071410
970,"(인삼류, 발효미생물류, 비타민 및 무기질)","(식이섬유, 지방산)",0.028415,0.007935,0.001072,0.037736,4.755737,0.000847,1.030970,0.812824,"{'['인삼류', '발효미생물류', '비타민 및 무기질'] -> ['식이섬유', '...",1.051356,0.071410
881,"(지방산, 인삼류, 비타민 및 무기질)","(페놀류, 발효미생물류)",0.032811,0.007720,0.001179,0.035948,4.656227,0.000926,1.029280,0.811873,"{'['지방산', '인삼류', '비타민 및 무기질'] -> ['페놀류', '발효미생...",1.050549,0.074108
896,"(페놀류, 발효미생물류)","(지방산, 인삼류, 비타민 및 무기질)",0.007720,0.032811,0.001179,0.152778,4.656227,0.000926,1.141600,0.791343,"{'['지방산', '인삼류', '비타민 및 무기질'] -> ['페놀류', '발효미생...",1.050549,0.074108


In [8]:
"""
[기준설정]

- 향상도 = 1.25 
  => 0.25개 더 먹은 게 제품 1를 더 먹은 꼴이 될 거 같아서

"""

association_rules_df[(association_rules_df["lift"]>1.2) & (association_rules_df["support"]>=0.001)\
                      & (association_rules_df["confidence"]>=0.05)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,all-confidence,collective strength,cosine similarity
65,"(지방산, 비타민 및 무기질)",(아미노산 및 단백질),0.166524,0.077204,0.015762,0.094656,1.226052,0.002906,1.019277,0.221211,"{'['지방산', '비타민 및 무기질'] -> ['아미노산 및 단백질']': 0.0...",1.035027,0.139016
68,(아미노산 및 단백질),"(지방산, 비타민 및 무기질)",0.077204,0.166524,0.015762,0.204167,1.226052,0.002906,1.047300,0.199799,"{'['지방산', '비타민 및 무기질'] -> ['아미노산 및 단백질']': 0.0...",1.035027,0.139016
72,"(지방산, 발효미생물류)",(신규기능성식품),0.091250,0.140039,0.015762,0.172738,1.233502,0.002984,1.039527,0.208309,"{'['신규기능성식품'] -> ['지방산', '발효미생물류']': 0.1125574...",1.037611,0.139438
73,(신규기능성식품),"(지방산, 발효미생물류)",0.140039,0.091250,0.015762,0.112557,1.233502,0.002984,1.024010,0.220127,"{'['신규기능성식품'] -> ['지방산', '발효미생물류']': 0.1125574...",1.037611,0.139438
103,"(지방산, 발효미생물류, 비타민 및 무기질)",(신규기능성식품),0.049003,0.140039,0.009114,0.185996,1.328174,0.002252,1.056458,0.259818,"{'['신규기능성식품'] -> ['지방산', '발효미생물류', '비타민 및 무기질'...",1.031974,0.110024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,"(식이섬유, 인삼류)","(지방산, 발효미생물류, 비타민 및 무기질)",0.007506,0.049003,0.001072,0.142857,2.915286,0.000704,1.109497,0.661949,"{'['지방산', '발효미생물류', '비타민 및 무기질'] -> ['식이섬유', '...",1.027447,0.055910
988,"(당 및 탄수화물, 아미노산 및 단백질)",(지방산),0.002788,0.298199,0.001072,0.384615,1.289796,0.000241,1.140427,0.225312,"{'['지방산'] -> ['당 및 탄수화물', '아미노산 및 단백질']': 0.00...",1.002301,0.037189
994,"(식이섬유, 당 및 탄수화물)",(비타민 및 무기질),0.001287,0.603367,0.001072,0.833333,1.381139,0.000296,2.379798,0.276315,"{'['비타민 및 무기질'] -> ['식이섬유', '당 및 탄수화물']': 0.00...",1.002475,0.038483
995,"(식이섬유, 비타민 및 무기질)",(당 및 탄수화물),0.020802,0.034956,0.001072,0.051546,1.474606,0.000345,1.017492,0.328690,"{'['당 및 탄수화물'] -> ['식이섬유', '비타민 및 무기질']': 0.03...",1.013613,0.039764


In [4]:
df["수"] = df["섭취품목"].apply(lambda x: to_list(x))
df

KeyError: '섭취품목'

In [42]:
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

C:\Users\USER\anaconda3\envs\newenv\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(비타민 및 무기질),(식이섬유),0.603367,0.033455,0.020802,0.034477,1.030542,0.000617,1.001058,0.074721
1,(식이섬유),(비타민 및 무기질),0.033455,0.603367,0.020802,0.621795,1.030542,0.000617,1.048725,0.030662
2,(아미노산 및 단백질),(식이섬유),0.077204,0.033455,0.004396,0.056944,1.702128,0.001813,1.024908,0.447011
3,(식이섬유),(아미노산 및 단백질),0.033455,0.077204,0.004396,0.131410,1.702128,0.001813,1.062408,0.426778
4,(엽록소),(식이섬유),0.024341,0.033455,0.001823,0.074890,2.238535,0.001009,1.044789,0.567082


In [34]:
max_i = 4
for i, row in association_rules_df[association_rules_df["lift"]>=1.25].iterrows():
    print("Rule: " + list(row['antecedents'])[0] + " => " + list(row['consequents'])[0])
 
    print("Support: " + str(round(row['support'],2)))
 
    print("Confidence: " + str(round(row['confidence'],2)))
    print("Lift: " + str(round(row['lift'],2)))
    print("=====================================")
    if i==max_i:
        break

Rule: 발효미생물류 => 신규기능성식품
Support: 0.02
Confidence: 0.17
Lift: 1.25
Rule: 신규기능성식품 => 발효미생물류
Support: 0.02
Confidence: 0.11
Lift: 1.25
Rule: 발효미생물류 => 신규기능성식품
Support: 0.01
Confidence: 0.19
Lift: 1.35
Rule: 발효미생물류 => 비타민 및 무기질
Support: 0.01
Confidence: 0.1
Lift: 1.43
Rule: 비타민 및 무기질 => 발효미생물류
Support: 0.01
Confidence: 0.13
Lift: 1.43
Rule: 신규기능성식품 => 발효미생물류
Support: 0.01
Confidence: 0.06
Lift: 1.35
Rule: 아미노산 및 단백질 => 지방산
Support: 0.01
Confidence: 0.4
Lift: 1.37
Rule: 지방산 => 아미노산 및 단백질
Support: 0.01
Confidence: 0.02
Lift: 1.37
Rule: 비타민 및 무기질 => 지방산
Support: 0.01
Confidence: 0.37
Lift: 1.25
Rule: 지방산 => 비타민 및 무기질
Support: 0.01
Confidence: 0.02
Lift: 1.25
Rule: 신규기능성식품 => 페놀류
Support: 0.0
Confidence: 0.03
Lift: 1.31
Rule: 페놀류 => 신규기능성식품
Support: 0.0
Confidence: 0.18
Lift: 1.31
Rule: 아미노산 및 단백질 => 지방산
Support: 0.0
Confidence: 0.44
Lift: 1.49
Rule: 아미노산 및 단백질 => 신규기능성식품
Support: 0.0
Confidence: 0.18
Lift: 1.3
Rule: 지방산 => 아미노산 및 단백질
Support: 0.0
Confidence: 0.12
Lift: 1.56
Rule: 아미노산 및 단백질 =

In [77]:
association_rules_df[association_rules_df["lift"]>=1.25]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,all-confidence,collective strength,cosine similarity
18,(아미노산 및 단백질),(식이섬유),0.077204,0.033455,0.004396,0.056944,1.702128,0.001813,1.024908,0.447011,{'아미노산 및 단백질 -> 식이섬유': 0.056944444444444436},1.039804,0.086505
19,(식이섬유),(아미노산 및 단백질),0.033455,0.077204,0.004396,0.131410,1.702128,0.001813,1.062408,0.426778,{'아미노산 및 단백질 -> 식이섬유': 0.056944444444444436},1.039804,0.086505
22,(엽록소),(식이섬유),0.024341,0.033455,0.001823,0.074890,2.238535,0.001009,1.044789,0.567082,{'식이섬유 -> 엽록소': 0.05448717948717949},1.039467,0.063879
23,(식이섬유),(엽록소),0.033455,0.024341,0.001823,0.054487,2.238535,0.001009,1.031884,0.572430,{'식이섬유 -> 엽록소': 0.05448717948717949},1.039467,0.063879
36,(아미노산 및 단백질),(엽록소),0.077204,0.024341,0.004611,0.059722,2.453610,0.002732,1.037629,0.642002,{'아미노산 및 단백질 -> 엽록소': 0.05972222222222221},1.065589,0.106363
...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,"(발효미생물류, 인삼류, 신규기능성식품)","(비타민 및 무기질, 지방산)",0.006541,0.166524,0.001501,0.229508,1.378231,0.000412,1.081746,0.276239,{'비타민 및 무기질 -> 발효미생물류': 0.00901481004507405},1.005844,0.045486
830,"(비타민 및 무기질, 인삼류, 신규기능성식품)","(발효미생물류, 지방산)",0.008578,0.091250,0.001501,0.175000,1.917803,0.000718,1.101515,0.482711,{'발효미생물류 -> 비타민 및 무기질': 0.016451233842538188},1.016456,0.053656
834,"(발효미생물류, 지방산)","(비타민 및 무기질, 인삼류, 신규기능성식품)",0.091250,0.008578,0.001501,0.016451,1.917803,0.000718,1.008005,0.526625,{'발효미생물류 -> 비타민 및 무기질': 0.016451233842538188},1.016456,0.053656
839,"(인삼류, 신규기능성식품)","(발효미생물류, 비타민 및 무기질, 지방산)",0.021017,0.049003,0.001501,0.071429,1.457643,0.000471,1.024151,0.320701,{'발효미생물류 -> 인삼류': 0.030634573304157548},1.015091,0.046778


In [31]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcl
import pandas as pd
import numpy as np
 
from matplotlib.colors import LinearSegmentedColormap
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [44]:
support = association_rules_df['support']
confidence = association_rules_df['confidence']
 
h = 347
s = 1
v = 1
colors = [
    mcl.hsv_to_rgb((h/360, 0.2, v)),
    mcl.hsv_to_rgb((h/360, 0.55, v)),
    mcl.hsv_to_rgb((h/360, 1, v))
]
cmap = LinearSegmentedColormap.from_list('my_cmap',colors,gamma=2)
 
measures = ['lift', 'leverage', 'conviction', 
            'all-confidence', 'collective strength', 'cosine similarity']
 
fig = plt.figure(figsize=(15,10))
fig.set_facecolor('white')
for i, measure in enumerate(measures):
    ax = fig.add_subplot(320+i+1)
    if measure != 'all-confidence':
        scatter = ax.scatter(support,confidence,c=association_rules_df[measure],cmap=cmap)
    else:
        scatter = ax.scatter(support,confidence,c=association_rules_df['all-confidence'].map(lambda x: [v for k,v in x.items()][0]),cmap=cmap)
    ax.set_xlabel('support')
    ax.set_ylabel('confidence')
    ax.set_title(measure)
    
    fig.colorbar(scatter,ax=ax)
fig.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

NameError: name 'mcl' is not defined

In [61]:
association_rules(frequent_itemset, metric='confidence', min_threshold=0.01)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(비타민 및 무기질),(식이섬유),0.603367,0.033455,0.020802,0.034477,1.030542,0.000617,1.001058,0.074721
1,(식이섬유),(비타민 및 무기질),0.033455,0.603367,0.020802,0.621795,1.030542,0.000617,1.048725,0.030662
2,(비타민 및 무기질),(아미노산 및 단백질),0.603367,0.077204,0.045250,0.074996,0.971401,-0.001332,0.997613,-0.069099
3,(아미노산 및 단백질),(비타민 및 무기질),0.077204,0.603367,0.045250,0.586111,0.971401,-0.001332,0.958308,-0.030918
4,(비타민 및 무기질),(지방산),0.603367,0.298199,0.166524,0.275991,0.925527,-0.013399,0.969327,-0.168657
...,...,...,...,...,...,...,...,...,...,...
837,"(비타민 및 무기질, 신규기능성식품)","(발효미생물류, 지방산, 인삼류)",0.070770,0.017371,0.001501,0.021212,1.221137,0.000272,1.003925,0.194883
838,"(지방산, 인삼류)","(발효미생물류, 비타민 및 무기질, 신규기능성식품)",0.077847,0.025735,0.001501,0.019284,0.749334,-0.000502,0.993422,-0.266194
839,"(인삼류, 신규기능성식품)","(발효미생물류, 비타민 및 무기질, 지방산)",0.021017,0.049003,0.001501,0.071429,1.457643,0.000471,1.024151,0.320701
840,"(지방산, 신규기능성식품)","(발효미생물류, 비타민 및 무기질, 인삼류)",0.039567,0.028415,0.001501,0.037940,1.335215,0.000377,1.009901,0.261400


In [73]:
for _,row in association_rules_df.iterrows():
    ### if/then, 조건/결과 생성
    all_confidence_if = list(row['antecedents'])
    all_confidence_then = list(row['consequents'])
    # - 반대일 경우 순서 수정
    if row['antecedent support'] <= row['consequent support']:
        all_confidence_if = list(row['consequents'])
        all_confidence_then = list(row['antecedents'])
        print(all_confidence_if)

['비타민 및 무기질']
['비타민 및 무기질']
['비타민 및 무기질']
['비타민 및 무기질']
['비타민 및 무기질']
['비타민 및 무기질']
['비타민 및 무기질']
['비타민 및 무기질']
['비타민 및 무기질']
['아미노산 및 단백질']
['지방산']
['식이섬유']
['인삼류']
['식이섬유']
['당 및 탄수화물']
['발효미생물류']
['신규기능성식품']
['지방산']
['아미노산 및 단백질']
['인삼류']
['아미노산 및 단백질']
['아미노산 및 단백질']
['발효미생물류']
['신규기능성식품']
['지방산']
['지방산']
['지방산']
['지방산']
['발효미생물류']
['지방산']
['인삼류']
['페놀류']
['당 및 탄수화물']
['발효미생물류']
['신규기능성식품']
['인삼류']
['인삼류']
['발효미생물류']
['인삼류']
['발효미생물류']
['신규기능성식품']
['발효미생물류']
['신규기능성식품']
['발효미생물류']
['아미노산 및 단백질']
['비타민 및 무기질']
['비타민 및 무기질', '아미노산 및 단백질']
['지방산']
['비타민 및 무기질']
['비타민 및 무기질', '지방산']
['인삼류']
['비타민 및 무기질']
['비타민 및 무기질', '인삼류']
['당 및 탄수화물']
['식이섬유']
['비타민 및 무기질']
['비타민 및 무기질']
['발효미생물류']
['발효미생물류', '비타민 및 무기질']
['신규기능성식품']
['비타민 및 무기질']
['비타민 및 무기질', '신규기능성식품']
['지방산']
['비타민 및 무기질']
['비타민 및 무기질', '지방산']
['아미노산 및 단백질']
['비타민 및 무기질']
['비타민 및 무기질', '아미노산 및 단백질']
['인삼류']
['비타민 및 무기질']
['비타민 및 무기질', '인삼류']
['아미노산 및 단백질']
['비타민 및 무기질']
['비타민 및 무기질', '아미노산 및 단백질']
['아미노산 및 단백질']
['비타민 및 무기질']
['

In [ ]:
df